# Loop 60 Analysis: CV-LB Relationship and Next Steps

**Goal**: Analyze the CV-LB relationship and identify approaches that might CHANGE the relationship, not just improve CV.

**Key Insight from exp_062 (Bias Correction)**:
- Bias correction FAILED because the bias is NOT uniform across targets
- Product 2: predictions are 0.009 LOWER than actuals (negative bias)
- Product 3: predictions are 0.003 LOWER than actuals (negative bias)
- SM: predictions are 0.021 HIGHER than actuals (positive bias)
- Subtracting a constant hurts some targets while helping others

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# CV-LB data from 13 submissions
submissions = [
    {'exp': 'exp_000', 'cv': 0.011081, 'lb': 0.0982},
    {'exp': 'exp_001', 'cv': 0.012297, 'lb': 0.1065},
    {'exp': 'exp_003', 'cv': 0.010501, 'lb': 0.0972},
    {'exp': 'exp_005', 'cv': 0.01043, 'lb': 0.0969},
    {'exp': 'exp_006', 'cv': 0.009749, 'lb': 0.0946},
    {'exp': 'exp_007', 'cv': 0.009262, 'lb': 0.0932},
    {'exp': 'exp_009', 'cv': 0.009192, 'lb': 0.0936},
    {'exp': 'exp_012', 'cv': 0.009004, 'lb': 0.0913},
    {'exp': 'exp_024', 'cv': 0.008689, 'lb': 0.0893},
    {'exp': 'exp_026', 'cv': 0.008465, 'lb': 0.0887},
    {'exp': 'exp_030', 'cv': 0.008298, 'lb': 0.0877},
    {'exp': 'exp_041', 'cv': 0.009002, 'lb': 0.0932},
    {'exp': 'exp_042', 'cv': 0.014503, 'lb': 0.1147},
]

df = pd.DataFrame(submissions)
print(f'Total submissions: {len(df)}')
print(df)

In [ ]:
# Fit linear regression: LB = slope * CV + intercept
slope, intercept, r_value, p_value, std_err = stats.linregress(df['cv'], df['lb'])

print(f'\n=== CV-LB Linear Regression ===')
print(f'LB = {slope:.4f} * CV + {intercept:.4f}')
print(f'R-squared: {r_value**2:.4f}')
print(f'Intercept: {intercept:.4f}')
print(f'\nTarget LB: 0.0347')
print(f'Intercept > Target: {intercept > 0.0347}')

# Required CV to hit target
required_cv = (0.0347 - intercept) / slope
print(f'\nRequired CV to hit target: {required_cv:.6f}')
print(f'This is IMPOSSIBLE (negative CV)' if required_cv < 0 else f'This requires CV = {required_cv:.6f}')

In [ ]:
# Plot CV vs LB
plt.figure(figsize=(10, 6))
plt.scatter(df['cv'], df['lb'], s=100, c='blue', alpha=0.7, label='Submissions')

# Regression line
cv_range = np.linspace(0, 0.02, 100)
lb_pred = slope * cv_range + intercept
plt.plot(cv_range, lb_pred, 'r--', label=f'LB = {slope:.2f}*CV + {intercept:.4f} (R²={r_value**2:.2f})')

# Target line
plt.axhline(y=0.0347, color='green', linestyle=':', linewidth=2, label='Target (0.0347)')

# Intercept line
plt.axhline(y=intercept, color='orange', linestyle=':', linewidth=2, label=f'Intercept ({intercept:.4f})')

plt.xlabel('CV Score (MSE)')
plt.ylabel('LB Score (MSE)')
plt.title('CV vs LB Relationship - All 13 Submissions')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('/home/code/exploration/cv_lb_relationship.png', dpi=150)
plt.show()

print(f'\nKey Insight: The intercept ({intercept:.4f}) is ABOVE the target (0.0347).')
print(f'This means even with perfect CV (CV=0), we would still get LB = {intercept:.4f}.')
print(f'The target is MATHEMATICALLY UNREACHABLE with the current paradigm.')

In [ ]:
# Analyze residuals - are there any outliers?
df['predicted_lb'] = slope * df['cv'] + intercept
df['residual'] = df['lb'] - df['predicted_lb']

print('=== Residual Analysis ===')
print(df[['exp', 'cv', 'lb', 'predicted_lb', 'residual']].to_string())

print(f'\nMean residual: {df["residual"].mean():.6f}')
print(f'Std residual: {df["residual"].std():.6f}')
print(f'Max residual: {df["residual"].max():.6f} ({df.loc[df["residual"].idxmax(), "exp"]})')
print(f'Min residual: {df["residual"].min():.6f} ({df.loc[df["residual"].idxmin(), "exp"]})')

In [ ]:
# What would it take to reach the target?
print('\n=== What Would It Take to Reach Target? ===')
print(f'Target LB: 0.0347')
print(f'Current intercept: {intercept:.4f}')
print(f'Gap: {intercept - 0.0347:.4f}')

print(f'\nOption 1: Reduce intercept by {intercept - 0.0347:.4f}')
print(f'  - This requires changing the CV-LB RELATIONSHIP, not just improving CV')
print(f'  - Approaches: Extrapolation detection, uncertainty weighting, domain constraints')

print(f'\nOption 2: Achieve CV = {required_cv:.6f}')
print(f'  - This is IMPOSSIBLE (negative CV)')

print(f'\nOption 3: Find an approach that has a DIFFERENT CV-LB relationship')
print(f'  - The "mixall" kernel uses GroupKFold (5 splits) - we tried this (exp_042) and it was WORSE')
print(f'  - The "Ens Model" kernel uses CatBoost + XGBoost - we tried CatBoost (exp_045) and it was WORSE')
print(f'  - Need to find something fundamentally different')

In [ ]:
# What approaches haven't been tried?
print('\n=== Approaches NOT Yet Tried ===')
print('\n1. TARGET-SPECIFIC BIAS CORRECTION')
print('   - exp_062 showed bias is NOT uniform across targets:')
print('     - Product 2: -0.009 (predictions too low)')
print('     - Product 3: -0.003 (predictions too low)')
print('     - SM: +0.021 (predictions too high)')
print('   - Instead of subtracting a constant from ALL predictions,')
print('     ADD to Product 2 and Product 3, SUBTRACT from SM')

print('\n2. ENSEMBLE VARIANCE AS UNCERTAINTY')
print('   - exp_058 used GP uncertainty, which is uniformly HIGH')
print('   - Instead, use variance across ensemble members')
print('   - When variance is high, blend toward population mean')

print('\n3. SOLVENT CLUSTERING + CLASS-SPECIFIC MODELS')
print('   - exp_052 tried per-solvent-type models and FAILED')
print('   - But we could try CLUSTERING solvents by chemical similarity')
print('   - Then use cluster-specific calibration')

print('\n4. PSEUDO-LABELING / SELF-TRAINING')
print('   - Use confident predictions on test data to augment training')
print('   - This could help with distribution shift')

print('\n5. DIFFERENT VALIDATION SCHEME')
print('   - The server might use a different CV scheme than Leave-One-Solvent-Out')
print('   - We tried GroupKFold (exp_042) and it was WORSE')
print('   - But maybe the server uses something else entirely')

In [ ]:
# Analyze the best experiments
print('\n=== Best Experiments Analysis ===')
best_cv = df.loc[df['cv'].idxmin()]
best_lb = df.loc[df['lb'].idxmin()]

print(f'Best CV: {best_cv["exp"]} with CV={best_cv["cv"]:.6f}, LB={best_cv["lb"]:.4f}')
print(f'Best LB: {best_lb["exp"]} with CV={best_lb["cv"]:.6f}, LB={best_lb["lb"]:.4f}')

print(f'\nBest CV (exp_030) predicted LB: {slope * 0.008298 + intercept:.4f}')
print(f'Actual LB: 0.0877')
print(f'Difference: {0.0877 - (slope * 0.008298 + intercept):.4f}')

print(f'\nexp_030 is BELOW the regression line (better than expected)')
print(f'This suggests exp_030 has some property that reduces the intercept')

In [ ]:
# What was special about exp_030?
print('\n=== What Was Special About exp_030? ===')
print('exp_030 was the GP + MLP + LGBM ensemble with weights (0.15, 0.55, 0.30)')
print('It achieved the best LB (0.0877) with CV 0.008298')
print('')
print('Key features:')
print('1. GP provides uncertainty estimates (but we found these are uniformly high)')
print('2. MLP captures non-linear patterns')
print('3. LGBM provides robustness')
print('4. Ensemble averaging reduces variance')
print('')
print('The ensemble approach seems to be the best we have.')

In [ ]:
# Calculate what improvement is needed
print('\n=== Required Improvement ===')
current_best_lb = 0.0877
target_lb = 0.0347
gap = current_best_lb - target_lb
pct_improvement = gap / current_best_lb * 100

print(f'Current best LB: {current_best_lb}')
print(f'Target LB: {target_lb}')
print(f'Gap: {gap:.4f}')
print(f'Required improvement: {pct_improvement:.1f}%')

print(f'\nThis is a MASSIVE gap. We need to reduce LB by 60%.')
print(f'No amount of CV optimization can achieve this.')
print(f'We need a fundamentally different approach.')

In [ ]:
# Summary
print('\n' + '='*60)
print('SUMMARY: LOOP 60 ANALYSIS')
print('='*60)
print(f'\n1. CV-LB Relationship: LB = {slope:.2f} * CV + {intercept:.4f} (R²={r_value**2:.2f})')
print(f'2. Intercept ({intercept:.4f}) > Target (0.0347) - IMPOSSIBLE to reach target by improving CV')
print(f'3. Best LB: 0.0877 (exp_030) - 152.7% above target')
print(f'4. Best CV: 0.008194 (exp_032) - but this doesn\'t translate to better LB')
print(f'\nKEY INSIGHT: The target IS reachable (someone achieved it).')
print(f'But not with our current paradigm.')
print(f'\nWe need to find an approach that CHANGES the CV-LB relationship,')
print(f'not just improves CV.')